## 1.Importing Libraries

In [ ]:
#To work with nd-arrays
import numpy

#To plot graph
%matplotlib inline
import matplotlib.pyplot as plt

#To use operating system dependent functionality
import os 

#To build neural network models
import keras

#Deep learning models with pre-trained weights
from keras import applications 

#To perform image augmentation-> scaling, rotation...
from keras.preprocessing.image import ImageDataGenerator 

#To minimize loss function
from keras import optimizers 

#Iporting different models in keras
from keras.models import Sequential, Model 

#To reshape the layers of the model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D 

#To import (three) backend implementations of keras
from keras import backend as k 

#To fix bugs in the model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping 

#To ignore warnings
import warnings
warnings.filterwarnings('ignore')

#For reproducibility
numpy.random.seed(1) # NumPy
import random
random.seed(2) # Python
from tensorflow import set_random_seed
set_random_seed(3) # Tensorflow

## 2. Load Test and Train Files

In [ ]:
files_train = 0
files_validation = 0

#To get current working directory
cwd = os.getcwd()
#Setting the path pointing to the training set
folder = 'training_data/train'

#To count number of files in training set
for sub_folder in os.listdir(folder):
    #os.path.join concatenates path by appropirately adding '/'
    root, dirs, files = next(os.walk(os.path.join(folder,sub_folder)))
    files_train += len(files)

#To count number of files in validation set
folder = 'training_data/validate'
for sub_folder in os.listdir(folder):
    #os.path.join concatenates path by appropirately adding '/'
    root, dirs, files = next(os.walk(os.path.join(folder,sub_folder)))
    files_validation += len(files)

#Total number of training files and validation files
print(files_train,files_validation)

## 3.Setting Key Parameters

In [ ]:
#Setting image height and width
img_width, img_height = 48, 48
#Setting training and validation directories
train_data_dir = "training_data/train"
validation_data_dir = "training_data/validate"
#Setting number of training and validation samples
nb_train_samples = files_train
nb_validation_samples = files_validation
#Number of samples used per pass
batch_size = 64
#1 epoch-> 1 complete forward and back pass
epochs = 100
#We have 2 classes only-> EMPTY and OCCUPIED
num_classes = 2

## 4. Build model on top of a trained VGG

In [ ]:
#Using VGG16 architecture - 16 layer model
    #imagenet-> pre-defined model/project to train and assign weights also provides annotations for objects in an image
    #input_shape-> should have exactly 3 input channels(R,G,B)
    #include_shape=False-> when we use pretrained models
model = applications.VGG16(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
    #Freezing layers will reduce computaional time
    #Dataset is small and data similarity is also less.
    #We need to train inly the basic features which will be same for all images
    #So we freeze top 5 layers
for layer in model.layers[:5]:
    layer.trainable = False
#To show the training status of each layer
for layer in model.layers:
    print(layer,layer.trainable)

## 5.Reshape output and compile the model

In [ ]:
x = model.output

#__________________________________________________________NOTE:______________________________________________________________
#There are different types of layers like dense, drop
    #dense-> fully connected layers, all nodes are connected to every other nodes in subsequent layer
      #We use 16 as output dimension for dense-> 2 classes*8 = 16
    #drop-> during training some random number of nodes will be dropped to prevent overfitting
    #flatten-> reshapes output equal to number of elements in input
        #flatten  -> (5,3)->(1,15)
        #dense(16)-> (1,15)->(1,16)
#__________________________________________________________NOTE:______________________________________________________________       


x = Flatten()(x)
#converting flattened layer model into dense(2)
predictions = Dense(num_classes, activation="softmax")(x)

# creating the final model
model_final = Model(input = model.input, output = predictions)

# compile the model
    #use categorical_crossentropy as model returns soft probabilities for each sample
    #Stochastic gradient boosting-> optimizer used while compiling the model
        #lr-> learning rate
        #momentum-> to smoothen update; should be high to give more weightage to past gradients - faster convergence
        #decay-> to lessen the learning rate gradually to find the optimal result
            #decay formula-> lr = self.lr * (1. / (1. + decay * epoch))
    #mean absolute error is used as metric to computer errors
lr0=0.1
decay_rate = lr0/epochs
model_final.compile(loss = "categorical_crossentropy", 
                optimizer = optimizers.SGD(lr=lr0, momentum=0.9, decay=decay_rate, nesterov=True), metrics=["mae","accuracy"]) 

## 6.Build Image data generators

In [ ]:
# Initiate the train and validation generators with data Augumentation
train_datagen = ImageDataGenerator(
#To scale RGB values in range of 0 to 1
rescale = 1./255,
#to flip input iage horizontally-> efficient augmentation
horizontal_flip = True,
#points outside boundaries are filled according similar to nearest one to the boundary
fill_mode = "nearest",
#to zoom 
zoom_range = 0.1,
#to shift image hoizontally (0.1 fraction of entire width)
width_shift_range = 0.1,
#to shift image vertically (0.1 fraction of entire height)
height_shift_range=0.1,
#Degree range for random rotations
rotation_range=5)

validation_datagen = ImageDataGenerator(
#To scale RGB values in range of 0 to 1
rescale = 1./255,
#to flip input iage horizontally-> efficient augmentation
horizontal_flip = True,
#points outside boundaries are filled according similar to nearest one to the boundary
fill_mode = "nearest",
#to zoom 
zoom_range = 0.1,
#to shift image hoizontally (0.1 fraction of entire width)
width_shift_range = 0.1,
#to shift image vertically (0.1 fraction of entire height)
height_shift_range=0.1,
#Degree range for random rotations
rotation_range=5)

#Setting the source and specifications for the train_generator to work on
train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size,
class_mode = "categorical")

#Setting the source and specifications for the test_generator to work on
validation_generator = validation_datagen.flow_from_directory(
validation_data_dir,
target_size = (img_height, img_width),
class_mode = "categorical")

## 7. Saving the model

In [ ]:
checkpoint = ModelCheckpoint("SK7_parking_lot.h5", #h5py file format to store results
                             monitor='val_acc', #accuracy computed on validaiton dataset
                             verbose=1, #logs are dispalyed
                             save_best_only=True, #Latest best model will not be overwritten 
                             save_weights_only=False, #If false full model will be saved. else only weights are saved
                             mode='auto', #If auto, automatically inferred from monitor method
                             period=1) #No of epochs between each checkpoint

early = EarlyStopping(monitor='val_acc', #accuracy computed on validaiton dataset
                      min_delta=0, #if subsequent iteration has no change in quantity, then considered as no improvement 
                      patience=2, #no of consecutive no improvement iterations allowed
                      verbose=1, #logs are dispalyed
                      mode='auto') #If auto, automatically inferred from monitor method


## 8.Training the model

In [ ]:
SK7_CNN_model = model_final.fit_generator(
train_generator, #Training data to be used is set using train_generator built
    
    #To mark the beginnning of next epoch, since keras is not aware of epoch
    #Keras function is loop infinity, so it is not aware of the begining and end of epoch    
steps_per_epoch = nb_train_samples//batch_size, 
    
epochs = epochs, #Number of epochs
validation_data = validation_generator, #Validation data to be used is set using validation_generator built

workers=1,
use_multiprocessing=False,
callbacks = [checkpoint, early]) #To save model and constraint to stop training

## 9.Plotting graph for the results

In [ ]:
print(SK7_CNN_model.history.keys())
plt.plot(SK7_CNN_model.history['acc'])
plt.plot(SK7_CNN_model.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.show()

In [ ]:
plt.plot(SK7_CNN_model.history['loss'])
plt.plot(SK7_CNN_model.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.show()